In [1]:
import warnings
import pandas as pd
import json
import re
from collections import Counter
from sklearn.model_selection import train_test_split
import logging
datPath = '/root/autodl-tmp/BertClassNews/data/'

warnings.filterwarnings("ignore")

In [28]:


Trpath = datPath + 'train_cls-sample.txt'
Testpath = datPath + 'dev_cls-sample.txt'

# f = open(pathTr, 'r')
# train_val = f.read()

def read_process_dat(path):
    try:
        with open(path) as f:
            text = f.read()
    except UnicodeDecodeError:        
        with open(path, errors='ignore') as f:
            text = f.read()
    pattern = r'[\t\n]'
    result = re.split(pattern, text)
    contentIndx = 0
    lblIndx = 1
    n = len(result) - 1
    content, label = [], []
    while lblIndx < n:
        content.append(result[contentIndx])
        label.append(result[lblIndx])
        contentIndx += 2
        lblIndx += 2
    df = pd.DataFrame()
    df['tag'] = label
    df['content'] = content
    df['len'] = df['content'].apply(lambda x: sum([i.isalpha() for i in x]))
    df = df[df['len']>27]
    return df

train_val_df = read_process_dat(Trpath)
test_df = read_process_dat(Testpath)

In [33]:
train_val_df.shape, test_df.shape, Counter(train_val_df['tag'].tolist()), Counter(test_df['tag'].tolist())

((32751, 3),
 (4532, 3),
 Counter({'政策类型': 2557,
          '基建民生': 6398,
          '社会热点': 8704,
          '人事任免': 2167,
          '产业金融': 1135,
          '本地旅游': 6880,
          '通报查处': 3097,
          '数据排名': 799,
          '暖新闻': 680,
          '人文历史': 334}),
 Counter({'数据排名': 111,
          '社会热点': 1205,
          '基建民生': 889,
          '本地旅游': 936,
          '政策类型': 370,
          '人事任免': 280,
          '通报查处': 436,
          '暖新闻': 95,
          '产业金融': 171,
          '人文历史': 39}))

In [32]:
# train valid split
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['tag'], random_state=42)
train_df.to_json(datPath + 'train_cls.json')
val_df.to_json(datPath + 'val_cls.json')
test_df.to_json(datPath + 'test_cls.json')

# Read data

In [2]:
import pandas as pd
datPath = '/root/autodl-tmp/BertClassNews/data/'
train_df = pd.read_json(datPath + 'train_cls.json')
val_df = pd.read_json(datPath + 'val_cls.json')
test_df = pd.read_json(datPath + 'test_cls.json')

# train_df = pd.read_json(datPath + 'smpl_train.json')
# val_df = pd.read_json(datPath + 'smpl_val.json')
# test_df = pd.read_json(datPath + 'smpl_test.json')

lblEncode = {j:i for i,j in zip(range(len(train_df['tag'].unique())), train_df['tag'].unique())}
train_df['tag'] = train_df['tag'].map(lblEncode)
val_df['tag'] = val_df['tag'].map(lblEncode)
test_df['tag'] = test_df['tag'].map(lblEncode)

In [3]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# Load model

In [4]:
# !pip install torch transformers
from transformers import BertTokenizer, BertForSequenceClassification

# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# lblEncode = {j:i for i,j in zip(range(len(train_df['tag'].unique())), train_df['tag'].unique())}

model = BertForSequenceClassification.from_pretrained('bert-base-chinese',num_labels=len(lblEncode))
# inputs = tokenizer(text, return_tensors='pt', padding=True)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
reverse_lblEncode = {lblEncode[i]:i for i in lblEncode}

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import sklearn.model_selection
from torch.nn import CrossEntropyLoss

In [7]:
reverse_lblEncode

{0: '本地旅游',
 1: '通报查处',
 2: '基建民生',
 3: '社会热点',
 4: '暖新闻',
 5: '人事任免',
 6: '政策类型',
 7: '产业金融',
 8: '人文历史',
 9: '数据排名'}

In [8]:
# reverse_lblEncode = {lblEncode[i]:i for i in lblEncode}

class BERTDataset(Dataset):

    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_texts, val_texts, test_texts = train_df['content'].tolist(), val_df['content'].tolist(), test_df['content'].tolist()
train_labels, val_labels, test_labels = train_df['tag'].tolist(), val_df['tag'].tolist(), test_df['tag'].tolist()

# train_texts, val_texts, test_texts = train_df['content'].iloc[:100].tolist(), val_df['content'].iloc[:100].tolist(), test_df['content'].iloc[:100].tolist()
# train_labels, val_labels, test_labels = train_df['tag'].iloc[:100].tolist(), val_df['tag'].iloc[:100].tolist(), test_df['tag'].iloc[:100].tolist()
def generateDataloader(df, lbl, BATCH_SIZE = 16):
    inputs = tokenizer(df, padding=True, truncation=True,return_tensors='pt')
    dataset = BERTDataset(inputs, lbl)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
    return dataloader
import time
import numpy as np

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
def evaluate(model, dataloader):
    model.eval()
    final_pred = []
    total = 0
    correct = 0
    pred = {i:[] for i in range(len(lblEncode))}
    # pred = {i:[] for i in [0,2,3]}
    with torch.no_grad():

        for batch in dataloader:
            inputs = torch.stack([t for t in batch['input_ids']]).to(device)
            labels = torch.tensor(batch['labels']).to(device)
            attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
            outputs = model(input_ids=inputs, attention_mask=attention_mask,
                      labels=labels)
            predictions = outputs[1].argmax(dim=1)
            for i in range(len(predictions)):
                predRes = predictions[i].item()
                pred[predRes].append((predictions[i] == labels[i]).item())
                final_pred.append((reverse_lblEncode[predRes], reverse_lblEncode[labels[i].item()]))
                final_pred.append((predRes, labels[i].item()))

            correct += (predictions == labels).sum().item()
            total += labels.shape[0]
      
    accuracy = correct/total
    res = {reverse_lblEncode[i]:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }
  # res = {i:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }

    return accuracy, res, final_pred

def model_train(model, train_dataloader, val_dataloader, iter = 3):
    for epoch in range(3):
      start = time.time()
      losses = 0
      model.train()
      idx = 0
      for batch in train_dataloader:
        inputs, token_type_ids, attention_mask, labels = batch['input_ids'], batch['token_type_ids'], batch['attention_mask'], batch['labels']

        input_ids = torch.stack([t for t in batch['input_ids']]).to(device)

        attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
        labels = torch.tensor(batch['labels']).to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,labels=labels)
        loss = criterion(outputs['logits'],labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses += loss.item()
        acc = (outputs['logits'].argmax(1) == labels).float().mean()
        idx += 1
        curr = time.time()
        timeConsume = curr - start
        if idx % 300 == 0:
            print (f'Epoch {epoch+1}, batch {idx}, so far takes {timeConsume}')
          # los  =outputs['loss']
        print (f'Loss: {losses}')

      print (acc)
      val_acc, val_cat_acc, val_final_pred= evaluate(model, val_dataloader)
      print ('val acc:', val_acc)
        # if idx % 10 == 0:
        #     logging.info(f"Epoch: {epoch}, Batch[{idx}/{len(train_iter)}], "
        #                   f"Train loss :{loss.item():.3f}, Train acc: {acc:.3f}")

      print(f'Epoch {epoch+1} complete, so far takes {timeConsume}')
    return model


def save_model(model, datPath, model_name = 'BertOrigmodelAllDat_v0.pt'):
    # datPath = '/root/wt/'
    # model_save_path  = datPath + 'BertOrigmodelAllDat_v0.pt'
    model_save_path  = datPath + model_name
    if os.path.exists(model_save_path):
        loaded_paras = torch.load(model_save_path)
        model.load_state_dict(loaded_paras)
        logging.info("## 成功载入已有模型，进行追加训练......")
    else:
        torch.save(model.state_dict(), model_save_path)
    return model

In [ ]:
class 

In [39]:
model = save_model(model, datPath)

# Retrain on the new training set

## Prepare for new dataset

In [9]:
train_dataloader = generateDataloader(train_df['content'].tolist(), train_df['tag'].tolist()) 
val_dataloader = generateDataloader(val_df['content'].tolist(), val_df['tag'].tolist()) 
test_dataloader = generateDataloader(test_df['content'].tolist(), test_df['tag'].tolist()) 

In [63]:
train_accuracy, train_cat_acc, train_pred = evaluate(model, train_dataloader)
print("Train Accuracy:", train_accuracy, "Train Cat Accuracy:", train_cat_acc)

Train Accuracy: 0.08729007633587786 Train Cat Accuracy: {'本地旅游': 0.06707317073170732, '通报查处': 0.0, '基建民生': 0.061068702290076333, '社会热点': 0.6525522041763341, '暖新闻': 0.0, '人事任免': 0.038632045598480054, '政策类型': 0.08110578021475896, '产业金融': 0.040842696629213485, '人文历史': 0.0, '数据排名': nan}


In [ ]:
new_model = model_train(model, train_dataloader, val_dataloader)

Epoch 1, batch 300, so far takes 102.42019772529602
Loss: 208.79968886822462
Epoch 1, batch 600, so far takes 226.628732919693
Loss: 339.1886717826128
Epoch 1, batch 900, so far takes 422.81718039512634
Loss: 446.6454291306436
Epoch 1, batch 1200, so far takes 619.0887923240662
Loss: 550.2644704021513
Epoch 1, batch 1500, so far takes 815.4085488319397
Loss: 636.8789555840194
tensor(0.8750, device='cuda:0')
val acc: 0.8778812395054191
Epoch 1 complete, so far takes 905.3765459060669
Epoch 2, batch 300, so far takes 196.26690912246704
Loss: 95.35405998118222
